In [1]:
!pip install pandas
!pip install seaborn

import numpy as np
import pandas as pd
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
import string
import tarfile
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow import keras
import nltk
import pickle


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
!pip install textblob
!pip install nltk
!pip install emoji
!pip install tensorflow
!pip install keras
!pip install tensorflow pandas numpy scikit-learn nltk
!pip install spacy
!python -m spacy download en_core_web_sm


nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('brown')
nltk.download('punkt')

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2024-10-13 09:06:58.164861: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Step 2: Initialize Lemmatizer and Stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [8]:
# Try reading the CSV with different encodings
try:
    data = pd.read_csv("lem_dataset.csv", encoding='latin1')  # Change encoding if necessary
    print(data.head())
except UnicodeDecodeError as e:
    print("Error reading the file:", e)
  


                                             Reviews  Sentiment
0  this film be just brilliant cast location scen...          1
1  big hair big boob bad music and a giant safety...          0
2  this have to be one of the bad film of the 199...          0
3  the at storytelle the traditional sort many ye...          1
4  bad mistake of my life   I pick this movie up ...          0


In [13]:
data['Sentiment'] = data['Sentiment'].map({'positive': 1.0, 'negative': 0.0})
# Check the modified state of the DataFrame
print(data.head())
print(data['Sentiment'].dtype)


                                             Reviews  Sentiment
0  this film be just brilliant cast location scen...        NaN
1  big hair big boob bad music and a giant safety...        NaN
2  this have to be one of the bad film of the 199...        NaN
3  the at storytelle the traditional sort many ye...        NaN
4  bad mistake of my life   I pick this movie up ...        NaN
float64


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60662 entries, 0 to 60661
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Reviews    60662 non-null  object 
 1   Sentiment  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 948.0+ KB


In [11]:
# Remove rows where 'Sentiment' is NaN
data = data.dropna(subset=['Sentiment'])

# Display the modified DataFrame
data.info()  # This will show you the DataFrame's info after removal
# Save the cleaned DataFrame to a new CSV file
data.to_csv('edited_reviews1.csv', index=False, encoding='utf-8')

# Display the first few rows of the cleaned DataFrame
print(data.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60662 entries, 0 to 60664
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Reviews    60662 non-null  object 
 1   Sentiment  60662 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB
                                             Reviews  Sentiment
0  <START this film was just brilliant casting lo...        1.0
1  <START big hair big boobs bad music and a gian...        0.0
2  <START this has to be one of the worst films o...        0.0
3  <START the <UNK> <UNK> at storytelling the tra...        1.0
4  <START worst mistake of my life . i picked thi...        0.0


In [12]:
def clean_text(text):
    text = re.sub(r'<START\s*', '', text)  # Remove <START and any following whitespace
    text = re.sub(r'<UNK>', '', text)      # Remove <UNK>
    text = re.sub(r'\s+', ' ', text)       # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]+$', '', text) 
    return text.strip()                    # Strip leading/trailing whitespace

# Apply the function to the 'Reviews' column
data['Reviews'] = data['Reviews'].apply(clean_text)

# Display the modified DataFrame
data.head()


,Reviews,Sentiment
0,this film was just brilliant casting location ...,1.0
1,big hair big boobs bad music and a giant safet...,0.0
2,this has to be one of the worst films of the 1...,0.0
3,the at storytelling the traditional sort many ...,1.0
4,worst mistake of my life . i picked this movie...,0.0


In [13]:
# to correct spelling
from textblob import TextBlob
def correct_spelling(text):
    blob = TextBlob(text)
    return str(blob.correct())
    data = data.applymap(lambda x: correct_spelling(x) if isinstance(x, str) else x)
    data.to_csv('edited_reviews1.csv', index=False, encoding='utf-8')
data.head()


,Reviews,Sentiment
0,this film was just brilliant casting location ...,1.0
1,big hair big boobs bad music and a giant safet...,0.0
2,this has to be one of the worst films of the 1...,0.0
3,the at storytelling the traditional sort many ...,1.0
4,worst mistake of my life . i picked this movie...,0.0


In [14]:
#cleaning and lowercasing
def clean_text(text):
    if isinstance(text, str):
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Convert to lowercase
        text = text.lower()
    return text


# Apply the cleaning function to each column
for column in data.columns:
    data[column] = data[column].map(clean_text)

# Save the cleaned DataFrame to a new CSV file
data.to_csv('dataset1.csv', index=False, encoding='utf-8')

# Display the first few rows of the cleaned DataFrame
print(data.head())


                                             Reviews  Sentiment
0  this film was just brilliant casting location ...        1.0
1  big hair big boobs bad music and a giant safet...        0.0
2  this has to be one of the worst films of the 1...        0.0
3  the at storytelling the traditional sort many ...        1.0
4  worst mistake of my life  i picked this movie ...        0.0


In [ ]:
import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Load the dataset
data = pd.read_csv('dataset1.csv')  # Replace with your actual file name

# Check if the 'Reviews' column exists
if 'Reviews' not in data.columns:
    raise ValueError("The 'Reviews' column is not found in the dataset.")

# Define a function to lemmatize text
def lemmatize_text(text):
    if isinstance(text, str):  # Check if the input is a string
        doc = nlp(text)
        return ' '.join([token.lemma_ for token in doc])
    else:
        return text  # Return as is if not a string

# Apply the lemmatization function to the 'Reviews' column
data['Reviews'] = data['Reviews'].apply(lemmatize_text)

# Save the updated DataFrame to a new CSV file
data.to_csv('lemmatized_reviews_spacy.csv', index=False)

print("Lemmatization applied and saved to 'lemmatized_reviews_spacy.csv'.")


In [2]:
data = pd.read_csv("lem_dataset.csv", encoding='latin1')  # Change encoding if necessary
print(data.head())

                                             Reviews  Sentiment
0  this film be just brilliant cast location scen...          1
1  big hair big boob bad music and a giant safety...          0
2  this have to be one of the bad film of the 199...          0
3  the at storytelle the traditional sort many ye...          1
4  bad mistake of my life   I pick this movie up ...          0


In [ ]:
#tokenization
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Define stop words
stop_words = set(stopwords.words('english'))

def clean_and_tokenize(text):
    if isinstance(text, str):
        # Tokenize text
        tokens = word_tokenize(text)
        # Remove stop words
        tokens = [word for word in tokens if word not in stop_words]
    else:
        tokens = []
    return tokens

# Apply tokenization only to the 'Reviews' column
if 'Reviews' in data.columns:
    data['Reviews'] = data['Reviews'].map(clean_and_tokenize)

# Save the cleaned DataFrame to a new CSV file
data.to_csv('tokenized.csv', index=False, encoding='utf-8')

# Display the first few rows of the cleaned DataFrame
print(data.head())


In [ ]:
data.shape

In [ ]:
data.info()

In [91]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the data from the CSV file
data_path = 'tokenized.csv'
data = pd.read_csv(data_path)

# Assuming 'text' column contains the preprocessed reviews
reviews = data['Reviews'].tolist()

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=10000)  # Adjust num_words based on vocabulary size limit
tokenizer.fit_on_texts(reviews)

# Convert the text to sequences
sequences = tokenizer.texts_to_sequences(reviews)
print("First  sequences:", sequences[:1])

# Pad the sequences to ensure uniform length
maxlen = 100  # You can adjust maxlen as needed for your data
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

# Check the shape of the padded sequences
print(f"Padded Sequences Shape: {padded_sequences.shape}")

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Tokenizer saved to 'tokenizer.pickle'")


First  sequences: [[3, 419, 84, 768, 1127, 17, 337, 192, 20, 1029, 58, 36, 28, 562, 471, 402, 46, 59, 237, 38, 3744, 757, 1, 35, 96, 64, 1329, 3, 1546, 3041, 373, 3, 23, 419, 25, 1, 343, 3, 406, 242, 15, 201, 192, 14, 738, 5219, 402, 20, 686, 40, 486, 27, 26, 686, 3, 118, 8, 314, 30, 47, 52, 197, 36, 2728, 696, 419, 164, 305, 77, 671, 1, 19, 75, 36, 446, 72, 5220, 131, 3, 164, 402, 1687, 19, 131, 17, 1112, 190, 187, 42, 880]]
Padded Sequences Shape: (60662, 100)
Tokenizer saved to 'tokenizer.pickle'


In [41]:
print(data.head())

                                             Reviews  Sentiment
0  ['film', 'brilliant', 'cast', 'location', 'sce...          1
1  ['big', 'hair', 'big', 'boob', 'bad', 'music',...          0
2  ['one', 'bad', 'film', '1990', 'friend', 'I', ...          0
3  ['storytelle', 'traditional', 'sort', 'many', ...          1
4  ['bad', 'mistake', 'life', 'I', 'pick', 'movie...          0


In [42]:
# Check the column names to find the text column
print(data.columns)


Index(['Reviews', 'Sentiment'], dtype='object')


In [47]:
X = padded_sequences
y = data['Sentiment']
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Shape of X: (60662, 100)
Shape of y: (60662,)


In [44]:
assert not np.any(np.isnan(X_train)), "Training data contains NaN values"
assert not np.any(np.isinf(X_train)), "Training data contains infinite values"
assert not np.any(np.isnan(y_train)), "Training data contains NaN values"
assert not np.any(np.isinf(y_train)), "Training data contains infinite values"



In [52]:

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Define model
model = Sequential()

# Add layers
model.add(Embedding(input_dim=10000, output_dim=128))  # Define vocab_size and max_length
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # For binary classification
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [53]:

model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/30
759/759 ━━━━━━━━━━━━━━━━━━━━ 80s 99ms/step - accuracy: 0.7622 - loss: 0.4677 - val_accuracy: 0.8559 - val_loss: 0.3338
Epoch 2/30
759/759 ━━━━━━━━━━━━━━━━━━━━ 75s 99ms/step - accuracy: 0.8975 - loss: 0.2563 - val_accuracy: 0.8603 - val_loss: 0.3232
Epoch 3/30
759/759 ━━━━━━━━━━━━━━━━━━━━ 75s 99ms/step - accuracy: 0.9278 - loss: 0.1904 - val_accuracy: 0.8590 - val_loss: 0.4033
Epoch 4/30
759/759 ━━━━━━━━━━━━━━━━━━━━ 77s 102ms/step - accuracy: 0.9459 - loss: 0.1461 - val_accuracy: 0.8521 - val_loss: 0.4140
Epoch 5/30
759/759 ━━━━━━━━━━━━━━━━━━━━ 78s 103ms/step - accuracy: 0.9626 - loss: 0.1076 - val_accuracy: 0.8480 - val_loss: 0.5019
Epoch 6/30
759/759 ━━━━━━━━━━━━━━━━━━━━ 79s 104ms/step - accuracy: 0.9710 - loss: 0.0869 - val_accuracy: 0.8446 - val_loss: 0.5519
Epoch 7/30
759/759 ━━━━━━━━━━━━━━━━━━━━ 81s 107ms/step - accuracy: 0.9759 - loss: 0.0734 - val_accuracy: 0.8418 - val_loss: 0.5798
Epoch 8/30
759/759 ━━━━━━━━━━━━━━━━━━━━ 75s 98ms/step - accuracy: 0.9840 - loss: 0.052

In [54]:
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 100, 128)            │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 32)                  │          20,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,198,373 (16.02 MB)

 Trainable params: 1,399,457 (5.34 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,798,916 (10.68 MB)

In [55]:
model.save('hahaha.keras')
data.head()

,Reviews,Sentiment
0,"['film', 'brilliant', 'cast', 'location', 'sce...",1
1,"['big', 'hair', 'big', 'boob', 'bad', 'music',...",0
2,"['one', 'bad', 'film', '1990', 'friend', 'I', ...",0
3,"['storytelle', 'traditional', 'sort', 'many', ...",1
4,"['bad', 'mistake', 'life', 'I', 'pick', 'movie...",0


In [56]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('hahaha.keras')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [57]:
# Evaluate on test data
print(f"Shape of X_train: {X_train.shape}, Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}, Shape of y_test: {y_test.shape}")
test_loss, test_acc = loaded_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')




Shape of X_train: (48529, 100), Shape of y_train: (48529,)
Shape of X_test: (12133, 100), Shape of y_test: (12133,)
380/380 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.8392 - loss: 1.0825
Test Accuracy: 0.8347482085227966


In [58]:

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [59]:
# Predict sentiment on the test data
predictions = loaded_model.predict(X_test)

# # Keep predictions as float values
# print("Predicted Sentiment Scores:")
# for i, prediction in enumerate(predictions):
#     print(f"Review {i+1}  : Sentiment Score: {prediction[0]:.4f}")




380/380 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step


In [61]:
# Convert predictions to binary values (0 or 1)
predicted_classes = (predictions > 0.5).astype(int)
print(predicted_classes)




[[0]
 [1]
 [0]
 ...
 [1]
 [1]
 [0]]


In [62]:
print("Length of y_test:", len(y_test))
print("Length of predicted_classes:", len(predicted_classes))

Length of y_test: 12133
Length of predicted_classes: 12133


In [7]:
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Step 1: Load the saved tokenizer and model
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load your trained model (assuming it's saved as 'sentiment_model.keras')
loaded_model = load_model('hahaha.keras')

# Retrieve the word index
word_index = tokenizer.word_index

# Clean the word index (if necessary)
cleaned_word_index = {key.strip("'"): value for key, value in word_index.items()}

# Step 2: Input comment
new_comment = "i did not like the movie at all"

# Step 3: Tokenize the input comment manually
comment_tokens = new_comment.lower().split()  # Lowercasing and splitting the comment
comment_sequence = [cleaned_word_index.get(word, 0) for word in comment_tokens]  # Convert words to indices

print("Comment Sequence:", comment_sequence)

# Step 4: Pad the sequence
maxlen = 100  # This should be the same as used during training
padded_sequence = pad_sequences([comment_sequence], maxlen=maxlen)

print("Padded Sequence:", padded_sequence)

# Step 5: Apply the model to get the prediction
prediction = loaded_model.predict(padded_sequence)
print("Prediction:", prediction)

# Step 6: Interpret the output
sentiment = 'Positive' if prediction >= 0.5 else 'Negative'
print(f"Sentiment: {sentiment}")


Comment Sequence: [1, 0, 0, 7, 0, 2, 0, 0]
Padded Sequence: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 7 0 2 0 0]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step
Prediction: [[0.05941871]]
Sentiment: Negative


In [97]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print(f'Test Accuracy: {accuracy:.4f}')

# Print classification report
report = classification_report(y_test, predicted_classes)
print(report)

# Print confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
print("Confusion Matrix:")
print(cm)


Test Accuracy: 0.8347
              precision    recall  f1-score   support

           0       0.83      0.83      0.83      5959
           1       0.84      0.84      0.84      6174

    accuracy                           0.83     12133
   macro avg       0.83      0.83      0.83     12133
weighted avg       0.83      0.83      0.83     12133

Confusion Matrix:
[[4963  996]
 [1009 5165]]


In [6]:
print("Word Index:", tokenizer.word_index)  # This will show the mapping of words to their indices.


Word Index: {"'i'": 1, "'movie'": 2, "'film'": 3, "'one'": 4, "'see'": 5, "'make'": 6, "'like'": 7, "'good'": 8, "'get'": 9, "'well'": 10, "'time'": 11, "'character'": 12, "'go'": 13, "'watch'": 14, "'would'": 15, "'bad'": 16, "'story'": 17, "'even'": 18, "'think'": 19, "'really'": 20, "'show'": 21, "'scene'": 22, "'great'": 23, "'look'": 24, "'much'": 25, "'say'": 26, "'know'": 27, "'could'": 28, "'people'": 29, "'also'": 30, "'first'": 31, "'take'": 32, "'give'": 33, "'way'": 34, "'love'": 35, "'play'": 36, "'thing'": 37, "'come'": 38, "'find'": 39, "'end'": 40, "'man'": 41, "'life'": 42, "'seem'": 43, "'work'": 44, "'plot'": 45, "'actor'": 46, "'two'": 47, "'many'": 48, "'year'": 49, "'want'": 50, "'never'": 51, "'little'": 52, "'try'": 53, "'ever'": 54, "'still'": 55, "'feel'": 56, "'something'": 57, "'part'": 58, "'director'": 59, "'old'": 60, "'use'": 61, "'back'": 62, "'lot'": 63, "'real'": 64, "'funny'": 65, "'performance'": 66, "'act'": 67, "'guy'": 68, "'though'": 69, "'anoth